In [1]:
import os
import json
import math
import sys
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from accelerate import Accelerator
from config import Config
from audiomodel import AudioProcessing
from audiodataset import AudioDataset, TestDataset

def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def build_model(cfg):
        from audiocraft.models.loaders import load_compression_model, load_lm_model
        """Instantiate models and optimizer."""
        compression_model = load_compression_model('facebook/audiogen-medium', device=cfg.device)
        lm = load_lm_model('facebook/audiogen-medium', device=cfg.device)
        return compression_model, lm

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [2]:
prompts = [
    "toilet water, music",
    "horn sound, crash sound",
    "horn sound followed by crashing cars",
    "man gregorian chanting, inside knight's helm",
    "large metal spikeball, hitting ground",
    "heavy footsteps wrapped in chains",
    "robotic interal mechanism moving",
    "frog followed by woosh",
    "rain, frog",
    "fire, wood building falling",
    "gun sound and then child crying",
    "big, church bell",
    "morning alarm",
    "machine exploding, parts falling",
    "missile firing and exploding",
    "whoosh, ice",
    "dragon wings flapping",
    "biting an apple",
    "walking on the shallow water",
    "engine starting up",
    "violin, concert hall",
    "rolling dice",
    "wine glass falling",
    "printer printing paper",
    "laser gun",
    "multimedia, notification",
    "truck accelerating",
    "frying chicken in a oil",
    "running, basketball court",
    "lightning hits tree",
    "rifle reloading",
    "fart sound machine gun",
    "crickets chirping",
    "chainsaw cutting tree",
    "girl, whispering"
]

In [ ]:
from audiotools import AudioSignal
torch.cuda.empty_cache()

cfg = Config()
cfg.update(**{"prompts": [p for p in prompts for _ in range(3)]})

accelerator = Accelerator(gradient_accumulation_steps=cfg.gradient_accumulation_steps)
save_path = "./test_base22"
make_dir(save_path)
cfg.update(**{"save_path": save_path})

compression_model, lm = build_model(cfg)
model = AudioProcessing(cfg, lm)

test_dataset = TestDataset(cfg)
test_dataloader = DataLoader(test_dataset, batch_size=1)

model, compression_model = accelerator.prepare(model, compression_model)
model_path = os.path.join("./compare/base_19.pth")
model.load_state_dict(torch.load(model_path))

model.eval()
compression_model.eval()
if accelerator.is_main_process:
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_vae = accelerator.unwrap_model(compression_model)
    audio_num = 1
    for test_step, batch in tqdm(enumerate(test_dataloader)):
        gen_tokens, gen_audio = unwrapped_model.inference(batch, unwrapped_vae)
        prompt = batch[0]
        print(prompt)
        audio_filename = f"{prompt}_{audio_num}.wav"
        unwrapped_model.save_audio(gen_audio, audio_filename, cfg)
        # from IPython.display import Audio
        # display(Audio(data=gen_audio[0].detach().cpu().numpy(), rate=cfg.sample_rate))
        # AudioSignal(gen_audio[0].detach().cpu().numpy(), sample_rate=cfg.sample_rate).widget()
        audio_num += 1
        if audio_num > 5:
            audio_num = 1

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
1it [00:05,  5.46s/it]

toilet water, music


2it [00:10,  5.15s/it]

toilet water, music


3it [00:15,  5.14s/it]

...

robotic interal mechanism moving


20it [01:39,  4.99s/it]

robotic interal mechanism moving


21it [01:44,  4.95s/it]

robotic interal mechanism moving


In [3]:
from audiotools import AudioSignal
# del model
# del compression_model
torch.cuda.empty_cache()

cfg = Config()
cfg.update(**{"prompts": [p for p in prompts for _ in range(3)]})

accelerator = Accelerator(gradient_accumulation_steps=cfg.gradient_accumulation_steps)
save_path = "./test_total_444"
make_dir(save_path)
cfg.update(**{"save_path": save_path})

compression_model, lm = build_model(cfg)
model = AudioProcessing(cfg, lm)

test_dataset = TestDataset(cfg)
test_dataloader = DataLoader(test_dataset, batch_size=1)

model, compression_model = accelerator.prepare(model, compression_model)
model_path = os.path.join("./output_dir_total22/44.pth")
model.load_state_dict(torch.load(model_path))

model.eval()
compression_model.eval()
if accelerator.is_main_process:
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_vae = accelerator.unwrap_model(compression_model)
    audio_num = 1
    for test_step, batch in tqdm(enumerate(test_dataloader)):
        gen_tokens, gen_audio = unwrapped_model.inference(batch, unwrapped_vae)
        prompt = batch[0]
        print(prompt)
        audio_filename = f"{prompt}_{audio_num}.wav"
        unwrapped_model.save_audio(gen_audio, audio_filename, cfg)
        # from IPython.display import Audio
        # display(Audio(data=gen_audio[0].detach().cpu().numpy(), rate=cfg.sample_rate))
        # AudioSignal(gen_audio[0].detach().cpu().numpy(), sample_rate=cfg.sample_rate).widget()
        audio_num += 1
        if audio_num > 5:
            audio_num = 1

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
1it [00:06,  6.13s/it]

toilet water, music


2it [00:12,  6.09s/it]

toilet water, music


3it [00:17,  5.71s/it]

...

frog followed by woosh


23it [01:58,  5.01s/it]

frog followed by woosh


23it [02:03,  5.35s/it]


KeyboardInterrupt: 

In [4]:
from audiotools import AudioSignal
torch.cuda.empty_cache()

cfg = Config()
cfg.update(**{"prompts": [p for p in prompts for _ in range(3)]})

accelerator = Accelerator(gradient_accumulation_steps=cfg.gradient_accumulation_steps)
save_path = "./test_total_noise"
make_dir(save_path)
cfg.update(**{"save_path": save_path})

compression_model, lm = build_model(cfg)
model = AudioProcessing(cfg, lm)

test_dataset = TestDataset(cfg)
test_dataloader = DataLoader(test_dataset, batch_size=1)

model, compression_model = accelerator.prepare(model, compression_model)
model_path = os.path.join("./output_dir_total22/19.pth")
model.load_state_dict(torch.load(model_path))

model.eval()
compression_model.eval()
if accelerator.is_main_process:
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_vae = accelerator.unwrap_model(compression_model)
    audio_num = 1
    for test_step, batch in tqdm(enumerate(test_dataloader)):
        gen_tokens, gen_audio = unwrapped_model.inference(batch, unwrapped_vae)
        prompt = batch[0]
        print(prompt)
        audio_filename = f"{prompt}_{audio_num}.wav"
        unwrapped_model.save_audio(gen_audio, audio_filename, cfg)
        # from IPython.display import Audio
        # display(Audio(data=gen_audio[0].detach().cpu().numpy(), rate=cfg.sample_rate))
        # AudioSignal(gen_audio[0].detach().cpu().numpy(), sample_rate=cfg.sample_rate).widget()
        audio_num += 1
        if audio_num > 5:
            audio_num = 1

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
1it [00:05,  5.27s/it]

frog followed by woosh


2it [00:10,  5.07s/it]

frog followed by woosh


3it [00:15,  5.00s/it]

...

girl, whispering


83it [07:04,  4.99s/it]

girl, whispering


84it [07:09,  5.11s/it]

girl, whispering
